# Monte Carlo Sampling and MCMC


## Rejection Sampling


taken from [https://www.deep-teaching.org/notebooks/monte-carlo-simulation/sampling/exercise-sampling-rejection]

Rejection sampling is a simple and straightforward algorihtm to generate samples for distributions, which a hard or impossible to sample from, using a second enclosing distribution. Using the enclosing function we will sample points and accept them as sample points for our desired distribution if they lie under the desired distribution curve and otherwise reject them. 



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

%matplotlib inline


In [ ]:
size=40
sigma_square_1 = 4.0
sigma_square_2 = 4.0
mu_1, sigma_1 = -2.5,np.sqrt(sigma_square_1)
mu_2, sigma_2 = 3.5,np.sqrt(sigma_square_1)
prob_1 = 0.4

rv_1 = norm(loc = mu_1, scale = sigma_1)
rv_2 = norm(loc = mu_2, scale = sigma_2)
x_ = np.arange(-14, 16, .1)

p_green = lambda x: prob_1 * rv_1.pdf(x) + (1-prob_1) * rv_2.pdf(x)
plt.plot(x_, p_green(x_) , "g-",label='$p(x)$')

sigma_red,mu_red = 5. , 1.
q_red = norm(loc = mu_red, scale = sigma_red)

plt.plot(x_, q_red.pdf(x_) , "r-",label='$q(x)$')
plt.legend()

_ = plt.xlabel("x")

We want to sample from the green distribution $p$ above but unfortunately we have no way to do so. We will use the normal distribution $q$ to obtain a sample from $p$ using rejection sampling.

### Exercise

- Implement rejection sampling to get a sample from $p(x)$. 


Let's first hypothesize that we identified a value $M$ such that:
$
\forall x \in \mathbb{R}, p(x) < M\cdot q(x)
$

The algorithm for rejection sampling works as follow:

- Sample a value $x$ from $q$
- Draw an value $u$ using an uniform distribution $\mathcal{U}(0,1)$ and compare $u$ to $\frac{p(x) }{M\cdot q(x)}$
   - if $u < \frac{p(x) }{M\cdot q(x)}$ then accept the sample $x$
   - if $u \geq \frac{p(x) }{M\cdot q(x)}$ discard the sample.


You can use an histogram to verify that your sample is indeed approaching the $p$ distribution.



In [ ]:
## Your code can go here.

##TODO: value M ?!
def rejection_sampling(size):
    samples = []

    while len(samples) < size:
        x = np.random.normal(mu_red, sigma_red)
        u = np.random.uniform(0, 1)

        acceptance_prob = p_green(x) / q_red.pdf(x)

        if u < acceptance_prob:
            samples.append(x)
    
    return samples

samples = rejection_sampling(size)
plt.hist(samples, bins=20, density=True)
plt.show()


# Text decoding using Metropolis-Hasting

In this exercise, we will apply the Metropolis-Hastings algorith, to the problem of decoding encoded by substitution, assuming we known the language of the text. 

We assume also that we can model this language using bigrams (an order 1 Markov chain on the letters). More formally, this language is written over the alphabet $\Lambda$. For example, in English, $\Lambda$ contains upper and lower case letters, punctuation marks, numbers, etc... 

The bigram model is given by $\mu$ and $A$:
 - $\mu$ is the stationnary distribution of single letters probabilities
 - $A$ is the transition matrix between letters: it gives for each letter the probability of the next letter. 
 
This model can easily be estimated from a large corpus of text. An encoding (or decoding) function by substitution is a bijective function $\tau: \Lambda \rightarrow \Lambda$. 
If $T′$ is a text, the encoded text $T = \tau(T′)$ is obtained by replacing each letter $c$ of $T′$ by $\tau(c)$.

Our formal problem is thus: given and encoded text $T = (c_1, c_2, . . . . , c_{|T|})$ ($c_i \in \Lambda, \forall i$), retrieve the initial decoded text.

## Preliminaries


1. How do you compute the likelihood of a sequence $T = (c_1, c_2, \cdots, c_{|T|})$ using a bigram model of parameter $(\mu, A)$? 

We note the likelihood this way: 
$
L(T, \mu, A) = P(c_1, c_2, \ldots, c_{|T|}\mid \mu, A) 
$

One way to find the code would be to use simple Monte Carlo and sample encoding functions, using a distribution proportional to the likelihood of the decoded text.

2. How many encoding functions are there? Is it possible to sample?

As we cannot sample directly a encoding function we will use MCMC. This method only needs the to know sampling probabilities up to a factor, which is the case here.

We will thus construct a Markov chain on the space of all decoded text, by sampling decoding functions $\tau$. 
The Metropolis-Hastings method works as follow:
- Choose an initial state $\tau_0$ arbitrarily
- Repeat $N$ times the following steps:
    - Compute $\tau$ from $\tau_t$ by swapping exactly 2 letters $c_1$ and $c_2$ in the encoding (see below for details on the function) 
    - Accept the transition from $\tau_t$ to $\tau$ with the probability $\alpha(\tau_t, \tau)$ :
    $$
    \alpha(\tau_t, \tau) = \min \left( 1, \frac{L(\tau(T), \mu, A)\cdot M(\tau_t,\tau)} 
    {L(\tau_t(T), \mu, A)\cdot M(\tau,\tau_t)}\right)
    $$
This mean that ones draw $u \sim \mathcal{U}(0,1)$:
        - if $u < \alpha(\tau_t, \tau)$ then $\tau_{t+1} = \tau$
        - otherwise $\tau_{t+1} = \tau_{t}$


After a sufficient large number of of iterations, the decoding function is drawn according to the probabilities of the decoded text.

3. Show that the log probability of acceptation can be written as:
$$
\log \alpha(\tau_t, \tau) = \min(0, \mu(\tau(c_1)) ) +
\sum_{i=1}^{|T|} log A(\tau(c_{i-1}), \tau(c_{i})) 
- \mu(\tau_t(c_1)) 
- \sum_{i=1}^{|T|} log A(\tau_t(c_{i-1}), \tau_t(c_{i})) 
$$




## Project

1) Download the file `countWar.pkl` which contains the template of bigrams learned from Tolstoy's novel _War and Peace_. This text was chosen because it contains more than 3 million characters, which is long enough to learn a representative model of a language. Also download the `secret.txt` message to decode (or this one `secret2.txt`, same message coded differently in case you would have problems with the first file).

2) Execute the following code to load in python the bigram model and the secret message.

   - The variable `count` is a dictionary: for each letter of the novel _War and Peace_, it provides its number of occurrences in the novel.
   - The pattern of bigrams is described by the variables `mu` and `A`.
   - The variable `mu` is a vector that contains the initial probability distribution over the letters.
   - `A` is a matrix that gives the probabilities of a letter given another letter.
   - `secret` is a variable containing the message to be decoded. 

In [ ]:
import pickle as pkl

# Open the dictionary
with open("./countWar.pkl", 'rb') as f:
    (count, mu, A) = pkl.load(f, encoding='latin1')

with open("./secret2.txt", 'r') as f:
    secret = f.read()[0:-1] # -1 to suppress the line break
 
 

3) The decoding functions will be represented as a dictionary where the key and the stored value are both of type character (one letter is encoded/decoded into another letter). 
  
Write a function 
  ``` swapF : (char,char) dict -> (char,char) dict ``` <br>
which takes as argument a decoding function $\tau_t$ and returns a new decoding function $\tau$ constructed by swapping two letters $c_1$ and $c_2$ as described in the previous question:
 -  $\tau(c)=\tau_t(c)$ for any $c$ such as $c \ne c_1$ and $c \ne c_2$;
 -  $\tau(c_1)=\tau_t(c_2)$
 -  $\tau(c_2)=\tau_t(c_1)$. 


You can test your function with the following dictionary:

In [ ]:
import random

def swapF(taut):
    tau = taut.copy()
    
    values = list(tau.values())
    first_letter = random.choice(values)
    second_letter = random.choice(values)
    
    while first_letter == second_letter:
        second_letter = random.choice(values)

    for i in tau.keys():
        if tau[i] == first_letter:
            tau[i] = second_letter
        elif tau[i] == second_letter:
            tau[i] = first_letter

    return tau
    


In [ ]:
tau = {'a' : 'b', 'b' : 'c', 'c' : 'a', 'd' : 'd' }
print(swapF(tau))

4) Write a function `decrypt: string x (char,char) dict -> string` <br>
which, given a `mess` string and a decoding function `tau`, returns the string obtained by decoding `mess` by `tau`.

You can test your function with the following code:

In [ ]:
def decrypt(mess, tau):
    decoded_mess = ""
    for letter in mess:
        if letter in tau.keys():
            decoded_mess += tau[letter]
        else:
            decoded_mess += letter
    return decoded_mess

In [ ]:
tau = {'a' : 'b', 'b' : 'c', 'c' : 'a', 'd' : 'd' }
print(decrypt ("aabcd", tau ))
print(decrypt ( "dcba", tau ))

which will produce the following output:

```
>>> decrypt ("aabcd", tau )
 bbcad
>>> decrypt ("dcba", tau )
 dacb
```

5) Create a dictionary (a hash table) associating to each character its index in `mu` or `A`. The code is simply the following:

In [ ]:
chars2index = dict(zip(np.array(list(count.keys())), np.arange(len(count.keys()))))

`chars2index['a']` simply gives access to the index corresponding to the letter `a` in `mu` or `A`

If you prefer, you can also use the index file that has already been generated: `fileHash.pkl`

6) Write a function `logLikelihood: string x float np.array x float np.2D-array x (char,int) dict-> string` which, given a `mess` message (string), the arrays `mu` and `A` created in question 2 from `pickle` and the previous dictionary `chars2index`, returns the log-likelihood of the `mess` message with respect to the big diagram model `(mu, A)`.

You can test your function with the following code:

logLikelihood( "abcd", mu, A, chars2index )
logLikelihood( "dcba", mu, A, chars2index )

In [ ]:
def logLikelihood(mess,mu,A,chars2index):

    # Initialize the log-likelihood to the log-probability of the first character
    log_likelihood = np.log(mu[chars2index[mess[0]]])

    # Iterate over the rest of the characters in the message
    for i in range(1, len(mess)):
        # Get the indices of the current and previous characters
        prev_index = chars2index[mess[i - 1]]
        curr_index = chars2index[mess[i]]
        
        # Add the log-probability of the transition from the previous character to the current one
        log_likelihood += np.log(A[prev_index, curr_index])

    return log_likelihood

In [ ]:
print(logLikelihood( "abcd", mu, A, chars2index ))
print(logLikelihood( "dcba", mu, A, chars2index ))

which will produce the following output:
```
 >>> logLikelihood( "abcd", mu, A, chars2index )
 -24.600258560804818
 >>> logLikelihood( "dcba", mu, A, chars2index )
 -26.274828997400395
```

7) Code the Metropolis-Hastings method seen in TD as a function called `MetropolisHastings(mess, mu, A, tau, N, chars2index)` using the `swapF`, `decrypt` and `logLikelihood` functions:
  - The `mess` parameter is the coded message.
  - Parameters `mu` and `A` represent the bigram model.
  - The argument `tau` is the initial decoding function to start the Metropolis-Hastings algorithm.
  - The argument `N` is the maximum number of iterations of the algorithm.
  - The argument `chars2index` has already been seen. 

The method is a simple loop where we do the following steps:
  - draw a new decoding function `tau'` by applying swapF with the current decoding function tau as parameter
  - calculation of the log-likelihood of the decoded message thanks to `tau'`.
  - draw to accept or not to accept the transition to `tau'` given the the ratio of likelihoods
  - if the transition is accepted, save the decoded message with the highest likelihood. 

The function returns the most likely decoded message. You will display the log-likelihood each time it is improved and the decoded message is saved.

the function returns the most likely decoded message. You will display the log-likelihood each time it is improved and the corresponding decoded message so that you can observe the evolution of the algorithm.

In order to test your function, you can execute the following code:

In [ ]:

def MetropolisHastings(mess, mu, A, tau, N, chars2index):
    # Initialize variables to store the best decoded message and its likelihood
    best_decoded_message = decrypt(mess, tau)
    best_likelihood = logLikelihood(best_decoded_message, mu, A, chars2index)
    current_tau = tau
    # Main Metropolis-Hastings loop
    for i in range(N):
        # Draw a new decoding function tau' by applying swapF to the current decoding function tau
        tau_prime = swapF(current_tau)
        # Calculate the log-likelihood of the decoded message with tau'
        message_candidate = decrypt(mess, tau_prime)
        likelihood_prime = logLikelihood(message_candidate, mu, A,chars2index)

        # Draw to accept or reject the transition to tau' based on the ratio of likelihoods
        if np.random.uniform(0, 1) < np.exp(likelihood_prime - best_likelihood):
            # If the transition is accepted, update the best decoded message and likelihood
            best_decoded_message = mess
            best_likelihood = likelihood_prime

            # Display the log-likelihood each time it is improved
            print(f"Iteration {i + 1}/{N} - Log-Likelihood: {best_likelihood:.4f}")
            
            current_tau = tau_prime
            
    return best_decoded_message


In [ ]:
def identityTau(count):
    tau = {}
    for k in list(count.keys()):
        tau[k] = k
    return tau

message = MetropolisHastings( secret, mu, A, identityTau(count), 10000, chars2index)


In [ ]:
print(message)

Warning: this (silly) code doesn't work with `secret.txt`, only with `secret2.txt`!

7) To speed up the calculations, we'll start from a decoding function taking the frequencies of occurrence of the letters (i.e. the most frequent letter of the coded message will be decoded to the most frequent letter observed in Tolstoy's novel; then the second most frequent letter of the coded message will be decoded to the second most frequent letter observed in the novel; and so on...). You can use the following code to build such a decoding function, named here `tau_init`.

In [ ]:
# WARNING: mu = proba of the init characters, not the stationary proba.
# => find frequent characters = sort (count) !
# stationary character distribution
freqKeys = np.array(list(count.keys()))
freqVal = np.array(list(count.values()))
# character index: +freq => - freq in references
rankFreq = (-freqVal).argsort()

# secret message analysis: index of the most frequent => least frequent
keys = np.array(list(set(secret))) # all characters of secret2
rankSecret = np.argsort(-np.array([secret.count(c) for c in keys]))
# ATTENTION: 37 keys in secret, 77 in general... 
# Only the most frequent characters of mu are encoded, 
# so much the worse for the others.
# alignment of + freq in mu VS + freq in secret
tau_init = dict((keys[rankSecret[i]], freqKeys[rankFreq[i]]) for i in range(len(rankSecret)))

MetropolisHastings(secret, mu, A, tau_init, 50000, chars2index)


The message will normally be intelligible when you reach a log-likelihood of more than -3090. However, there are usually still some errors in the translation... At which level? 

